In [1]:
import sys
sys.path.append('..')
from src.finance_ml.backtests.strategy import Strategy, MarketAction, Operator
from src.finance_ml.data_preparation.data_preparation import DataLoader
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor



In [2]:
dataloader = DataLoader(time_index_col= 'DATE', keep_cols = ['VOLUME','OPEN', 'HIGHT', 'LOW', 'CLOSE', 'VW','TRANSACTIONS'])
# Example Files
fname_IGIB = 'data/fixed_income/IGIB_2020-04-07_2022-04-06.parquet'

# No. of Records from example dataset
N =-1
# Dataset chosen in this simulation
ticker = 'IGIB'
fname = fname_IGIB
# loading assets into to an unique df
df = dataloader.load_dataset({ticker: fname_IGIB}).iloc[:N]
display(df)


,IGIB_VOLUME,IGIB_OPEN,IGIB_HIGHT,IGIB_LOW,IGIB_CLOSE,IGIB_VW,IGIB_TRANSACTIONS
DATE,,,,,,,
2020-04-07 17:33:00,492.0,55.2935,55.2935,55.290,55.2900,55.2914,4
2020-04-07 17:34:00,5939.0,55.3050,55.3050,55.290,55.2902,55.2968,40
2020-04-07 17:35:00,6469.0,55.2915,55.3000,55.280,55.2850,55.2896,66
2020-04-07 17:36:00,3131.0,55.2900,55.2900,55.280,55.2900,55.2881,9
2020-04-07 17:37:00,1023.0,55.2900,55.3000,55.290,55.3000,55.2956,8
...,...,...,...,...,...,...,...
2022-04-06 19:55:00,714.0,53.9600,53.9600,53.950,53.9500,53.9584,5
2022-04-06 19:56:00,1557.0,53.9500,53.9650,53.950,53.9650,53.9538,17
2022-04-06 19:57:00,237.0,53.9650,53.9650,53.965,53.9650,53.9650,4


In [3]:
data = df.copy()

# Fetch and prepare data
data.reset_index(inplace=True)
data['Next_Close'] = data['IGIB_CLOSE'].shift(-1)
data['Day'] = data['DATE'].dt.day
data['Month'] = data['DATE'].dt.month
data['Year'] = data['DATE'].dt.year
#print(data.head())
#data = data[['Day', 'Month', 'Year', 'Next_Close','Hour','Minute']]
data.dropna(inplace=True)

# Separate features and target
X = data[['Day', 'Month', 'Year']]
y = data['Next_Close']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardize features for each feature to a mean of 0 and a standard deviation of 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train PLS Regression model
model = PLSRegression(n_components=1)
model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = model.predict(X_test_scaled)

# Display metrics
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(y_pred.shape)
print("Mean Squared Error:", mse)
print("R-squared:", r2)


(32871, 1)
Mean Squared Error: 1.8888014770986654
R-squared: 0.35936085302500953


In [4]:
y_pred = model.predict(X_test_scaled)

# Convert y_pred to a DataFrame and set the index to match X_test
y_pred_df = pd.DataFrame(y_pred, index=X_test.index, columns=['a_PNClose'])

# Join/merge the predicted values with the original dataset
data1 = data.join(y_pred_df, how='left')

# Now, data_with_predictions_clean will only contain rows that have a predicted closing value
print(y_pred_df)
print(data1['a_PNClose'])


        a_PNClose
134009  60.245016
145591  57.692998
88355   59.228788
140459  60.348500
100870  59.525052
...           ...
95341   59.390718
12084   60.299486
162004  58.016857
112624  59.742174
141822  57.630908

[32871 rows x 1 columns]
0         60.017020
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
164348    58.085652
164349    58.085652
164350          NaN
164351          NaN
164352          NaN
Name: a_PNClose, Length: 164353, dtype: float64


In [5]:
tdata = data1.copy()
tdata.columns = [c.split('_')[-1] for c in tdata.columns]
tdata = tdata.reset_index()

# make daily data
tdata = tdata.set_index('DATE').resample('D')
tdata = tdata.aggregate({'OPEN': 'first', 'HIGHT': 'max', 'LOW': 'min', 'CLOSE': 'last', 'VOLUME': 'sum', 'VW': 'mean', 'TRANSACTIONS': 'sum', 'PNClose': 'last'})
tdata = tdata.dropna()
tdata = tdata.reset_index()
print(tdata.head())

        DATE     OPEN    HIGHT    LOW    CLOSE      VOLUME         VW  \
0 2020-04-07  55.2935  55.5000  55.21  55.2500   1181834.0  55.290144   
1 2020-04-08  55.3500  55.9700  55.33  55.8600   1450054.0  55.722456   
2 2020-04-09  56.4600  57.5538  53.82  57.5538  24143297.0  56.760171   
3 2020-04-13  57.2600  57.5500  56.55  57.3401   4186936.0  57.307612   
4 2020-04-14  57.8200  57.8400  57.26  57.3100   2357930.0  57.516740   

   TRANSACTIONS    PNClose  
0          2863  60.017020  
1          4890  60.020469  
2         52768  60.023919  
3         15114  60.037716  
4         11466  60.041166  


In [6]:
strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
strategy, type(strategy)


(Strategy: Simple Strategy, strategy.Strategy)

In [7]:
data = tdata.copy()
# Define the threshold for considering a significant price change
price_change_threshold = 0.005  # 1% change, you can adjust this based on your strategy
# Preprocess data
data['Buy_Signal'] = (data['PNClose'] / data['CLOSE']) - 1 > price_change_threshold
data['Sell_Signal'] = (data['PNClose'] / data['CLOSE']) - 1 < -price_change_threshold


strategy.load_data(data, date_column='DATE')
strategy.data.head()

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS,PNClose,Buy_Signal,Sell_Signal
0,2020-04-07,55.2935,55.5000,55.21,55.2500,1181834.0,55.290144,2863,60.017020,True,False
1,2020-04-08,55.3500,55.9700,55.33,55.8600,1450054.0,55.722456,4890,60.020469,True,False
2,2020-04-09,56.4600,57.5538,53.82,57.5538,24143297.0,56.760171,52768,60.023919,True,False
3,2020-04-13,57.2600,57.5500,56.55,57.3401,4186936.0,57.307612,15114,60.037716,True,False
4,2020-04-14,57.8200,57.8400,57.26,57.3100,2357930.0,57.516740,11466,60.041166,True,False


In [8]:
true_counts = data[['Buy_Signal']].sum()
both_signals = (data['Buy_Signal'] == True) & (data['Sell_Signal'] == True)
data['both'] = both_signals
print(data['both'].sum())
print(true_counts)
true_counts2 = data[['Sell_Signal']].sum()
print(true_counts2)

0
Buy_Signal    146
dtype: int64
Sell_Signal    228
dtype: int64


In [9]:


# Create rules based on preprocessed data
buy_rule = strategy.add_single_rule('Buy_Signal[N-1]==1', action=MarketAction.BUY, action_quantity='ALL')
sell_rule = strategy.add_single_rule('Sell_Signal[N-1]==1', action=MarketAction.SELL, action_quantity='ALL')


In [10]:
strategy.simulate()

In [11]:
strategy.summary()

Strategy Name: Simple Strategy
First Trade Date: 2020-04-07 00:00:00
Last Trade Date: 2022-02-03 00:00:00
Total Trades: 7
Final PnL: 99.8851069634386
Starting Cash: 1000
Final Cash: 0.0008334090668995486
Final Positions: 19.140072627762493
Final Portfolio Value: 1099.8851069634386
Total Commission: 6.450423850254516
Total Buy Commission: 3.7037137408213203
Total Sell Commission: 2.7467101094331956
Total Buy Amount: 4315.1468999423305
Total Sell Amount: 3315.147733351397
Total Buy Quantity: 74.0742748164264
Total Sell Quantity: 54.9342021886639
Total Buy Trades: 4
Total Sell Trades: 3


In [12]:

# Define the range of threshold values to test
threshold_values = np.arange(0.001, 0.01, 0.001)  # Adjust the range and step as needed

# Initialize variables to store the best threshold and its corresponding profit
best_threshold = None
max_profit = 0

# Iterate over threshold values
for threshold in threshold_values:
    # Create a new instance of the strategy for each iteration
    strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
    
    # Preprocess data based on the current threshold
    data['Buy_Signal'] = (data['PNClose'] / data['CLOSE']) - 1 > threshold
    data['Sell_Signal'] = (data['PNClose'] / data['CLOSE']) - 1 < -threshold

    strategy.load_data(data, date_column='DATE')

    # Define rules
    strategy.add_single_rule('Buy_Signal[N-1]==1', action=MarketAction.BUY, action_quantity='ALL')
    strategy.add_single_rule('Sell_Signal[N-1]==1', action=MarketAction.SELL, action_quantity='ALL')

    # Simulate the strategy
    strategy.simulate()

    # Get final portfolio value
    # Check if any trades were made
    if not strategy.history_df.empty:
        # Get final portfolio value
        final_portfolio_value = strategy.history_df['portfolio_value'].iloc[-1]
    else:
        # If no trades were made, use the starting cash value as the final portfolio value
        final_portfolio_value = strategy._starting_cash

    # Check if this is the best threshold so far
    if final_portfolio_value > max_profit:
        max_profit = final_portfolio_value
        best_threshold = threshold

    print(f"Threshold: {threshold}, Final Portfolio Value: {final_portfolio_value}")

print(f"Best Threshold: {best_threshold}, Maximum Profit: {max_profit}")


Threshold: 0.001, Final Portfolio Value: 1093.7075739526967
Threshold: 0.002, Final Portfolio Value: 1088.670288356545
Threshold: 0.003, Final Portfolio Value: 1093.5548851453655
Threshold: 0.004, Final Portfolio Value: 1091.935604588959
Threshold: 0.005, Final Portfolio Value: 1099.8851069634386
Threshold: 0.006, Final Portfolio Value: 1102.029606128361
Threshold: 0.007, Final Portfolio Value: 1105.8085174867617
Threshold: 0.008, Final Portfolio Value: 1107.9268219853268
Threshold: 0.009000000000000001, Final Portfolio Value: 1114.7142517186448
Best Threshold: 0.009000000000000001, Maximum Profit: 1114.7142517186448


In [13]:
# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.009000000000000001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
data['Significant_Price_Change_After'] = (data['PNClose'] / data['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
data['Daily_Return_Before'] = data['CLOSE'].pct_change()
data['Daily_Return_After'] = data['CLOSE'].pct_change() * data['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Sort daily returns before and after transactions based on the best threshold
sorted_returns_before = data['Daily_Return_Before'].sort_values()
sorted_returns_after = data['Daily_Return_After'].sort_values()

# Calculate the index for the VaR and CVaR before transactions based on the best threshold
index_1_percent_before = int(0.01 * len(sorted_returns_before))
index_5_percent_before = int(0.05 * len(sorted_returns_before))

# Calculate VaR and CVaR before transactions based on the best threshold
var_1_percent_before = sorted_returns_before.iloc[index_1_percent_before]
cvar_1_percent_before = sorted_returns_before.iloc[:index_1_percent_before].mean()

var_5_percent_before = sorted_returns_before.iloc[index_5_percent_before]
cvar_5_percent_before = sorted_returns_before.iloc[:index_5_percent_before].mean()

# Calculate the index for the VaR and CVaR after transactions based on the best threshold
index_1_percent_after = int(0.01 * len(sorted_returns_after))
index_5_percent_after = int(0.05 * len(sorted_returns_after))

# Calculate VaR and CVaR after transactions based on the best threshold
var_1_percent_after = sorted_returns_after.iloc[index_1_percent_after]
cvar_1_percent_after = sorted_returns_after.iloc[:index_1_percent_after].mean()

var_5_percent_after = sorted_returns_after.iloc[index_5_percent_after]
cvar_5_percent_after = sorted_returns_after.iloc[:index_5_percent_after].mean()

# Display results
print("VaR (1%) before transactions (best threshold):", var_1_percent_before)
print("CVaR (1%) before transactions (best threshold):", cvar_1_percent_before)

print("VaR (5%) before transactions (best threshold):", var_5_percent_before)
print("CVaR (5%) before transactions (best threshold):", cvar_5_percent_before)

print("VaR (1%) after transactions (best threshold):", var_1_percent_after)
print("CVaR (1%) after transactions (best threshold):", cvar_1_percent_after)

print("VaR (5%) after transactions (best threshold):", var_5_percent_after)
print("CVaR (5%) after transactions (best threshold):", cvar_5_percent_after)


VaR (1%) before transactions (best threshold): -0.009872147596698522
CVaR (1%) before transactions (best threshold): -0.012902696338238307
VaR (5%) before transactions (best threshold): -0.005346700083542211
CVaR (5%) before transactions (best threshold): -0.008327159436293878
VaR (1%) after transactions (best threshold): -0.009306569343065618
CVaR (1%) after transactions (best threshold): -0.011863166591845897
VaR (5%) after transactions (best threshold): -0.002649240551041987
CVaR (5%) after transactions (best threshold): -0.006376246571669233


In [14]:
# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.009000000000000001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
data['Significant_Price_Change_After'] = (data['PNClose'] / data['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
data['Daily_Return_Before'] = data['CLOSE'].pct_change()
data['Daily_Return_After'] = data['CLOSE'].pct_change() * data['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Calculate mean return (short) before transactions
mean_return_short_before = data[data['Daily_Return_Before'] < 0]['Daily_Return_Before'].mean()

# Calculate mean return (long) before transactions
mean_return_long_before = data[data['Daily_Return_Before'] >= 0]['Daily_Return_Before'].mean()

# Calculate mean return (short) after transactions
mean_return_short_after = data[data['Daily_Return_After'] < 0]['Daily_Return_After'].mean()

# Calculate mean return (long) after transactions
mean_return_long_after = data[data['Daily_Return_After'] >= 0]['Daily_Return_After'].mean()

# Display results
print("Mean Return (Short) before transactions:", mean_return_short_before)
print("Mean Return (Long) before transactions:", mean_return_long_before)

print("Mean Return (Short) after transactions (best threshold):", mean_return_short_after)
print("Mean Return (Long) after transactions (best threshold):", mean_return_long_after)


Mean Return (Short) before transactions: -0.0025218856950098237
Mean Return (Long) before transactions: 0.0021452901037482684
Mean Return (Short) after transactions (best threshold): -0.003924056966997792
Mean Return (Long) after transactions (best threshold): 0.0005299063323948238


In [15]:
import numpy as np

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.009000000000000001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
data['Significant_Price_Change_After'] = (data['PNClose'] / data['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
data['Daily_Return_Before'] = data['CLOSE'].pct_change()
data['Daily_Return_After'] = data['CLOSE'].pct_change() * data['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Calculate statistics before transactions
stats_before = data['Daily_Return_Before'].describe()

# Calculate statistics after transactions
stats_after = data['Daily_Return_After'].describe()

# Display results
print("Statistics before transactions:")
print("Minimum:", stats_before['min'])
print("Quartile 1:", stats_before['25%'])
print("Median:", stats_before['50%'])
print("Quartile 3:", stats_before['75%'])
print("Maximum:", stats_before['max'])

print("\nStatistics after transactions (best threshold):")
print("Minimum:", stats_after['min'])
print("Quartile 1:", stats_after['25%'])
print("Median:", stats_after['50%'])
print("Quartile 3:", stats_after['75%'])
print("Maximum:", stats_after['max'])


Statistics before transactions:
Minimum: -0.016727272727272813
Quartile 1: -0.0013793745422527315
Median: 0.00016258849448158585
Quartile 3: 0.0016637018037399254
Maximum: 0.010556300268096619

Statistics after transactions (best threshold):
Minimum: -0.016727272727272813
Quartile 1: 0.0
Median: -0.0
Quartile 3: 0.0
Maximum: 0.010556300268096619


In [16]:
# Apply the best threshold to create a binary column for significant price changes after transactions
data['Significant_Price_Change_After'] = (data['PNClose'] / data['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
data['Daily_Return_Before'] = data['CLOSE'].pct_change()
data['Daily_Return_After'] = data['CLOSE'].pct_change() * data['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Calculate standard deviation before transactions
std_dev_before = data['Daily_Return_Before'].std()

# Calculate standard deviation after transactions based on the best threshold
std_dev_after = data['Daily_Return_After'].std()

# Display results
print("Standard Deviation before transactions:", std_dev_before)
print("Standard Deviation after transactions (best threshold):", std_dev_after)


Standard Deviation before transactions: 0.0031907795962179742
Standard Deviation after transactions (best threshold): 0.0021916089380974277


In [17]:
import numpy as np

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.009000000000000001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
data['Significant_Price_Change_After'] = (data['PNClose'] / data['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
data['Daily_Return_Before'] = data['CLOSE'].pct_change()
data['Daily_Return_After'] = data['CLOSE'].pct_change() * data['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Skewness before transactions
skewness_before = data['Daily_Return_Before'].skew()

# Kurtosis before transactions
kurtosis_before = data['Daily_Return_Before'].kurtosis()

# Calculate drawdowns before transactions
cumulative_returns_before = (1 + data['Daily_Return_Before']).cumprod()
running_max_before = cumulative_returns_before.cummax()
drawdown_before = (cumulative_returns_before / running_max_before) - 1
max_drawdown_before = drawdown_before.min()

# Calmar ratio before transactions
calmar_ratio_before = cumulative_returns_before.iloc[-1] / abs(max_drawdown_before)

# Skewness after transactions
skewness_after = data['Daily_Return_After'].skew()

# Kurtosis after transactions
kurtosis_after = data['Daily_Return_After'].kurtosis()

# Calculate drawdowns after transactions
cumulative_returns_after = (1 + data['Daily_Return_After']).cumprod()
running_max_after = cumulative_returns_after.cummax()
drawdown_after = (cumulative_returns_after / running_max_after) - 1
max_drawdown_after = drawdown_after.min()

# Calmar ratio after transactions
calmar_ratio_after = cumulative_returns_after.iloc[-1] / abs(max_drawdown_after)

# Display results
print("Skewness before transactions:", skewness_before)
print("Kurtosis before transactions:", kurtosis_before)
print("Maximum Drawdown before transactions:", max_drawdown_before)
print("Calmar Ratio before transactions:", calmar_ratio_before)

print("\nSkewness after transactions (best threshold):", skewness_after)
print("Kurtosis after transactions (best threshold):", kurtosis_after)
print("Maximum Drawdown after transactions (best threshold):", max_drawdown_after)
print("Calmar Ratio after transactions (best threshold):", calmar_ratio_after)


Skewness before transactions: -0.6898074585317826
Kurtosis before transactions: 3.4320522799877202
Maximum Drawdown before transactions: -0.12491909385113098
Calmar Ratio before transactions: 7.60708735239591

Skewness after transactions (best threshold): -1.309125244370409
Kurtosis after transactions (best threshold): 15.738160457820346
Maximum Drawdown after transactions (best threshold): -0.05991814405997142
Calmar Ratio after transactions (best threshold): 17.043808389510698


In [18]:
import statsmodels.api as sm

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.009000000000000001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
data['Significant_Price_Change_After'] = (data['PNClose'] / data['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
data['Daily_Return_Before'] = data['CLOSE'].pct_change()
data['Daily_Return_After'] = data['CLOSE'].pct_change() * data['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Calculate Newey-West standard error, t-statistic (Newey-West), and PT test statistics before transactions
X_before = sm.add_constant(data['Daily_Return_Before'].shift(1))
model_before = sm.OLS(data['Daily_Return_Before'], X_before, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 1})
nw_standard_error_before = np.sqrt(model_before.cov_params().iloc[1, 1])
t_stat_before = model_before.tvalues[1]
pt_stat_before = sm.stats.acorr_ljungbox(model_before.resid, lags=[1], return_df=True)['lb_stat'][1]

# Calculate Newey-West standard error, t-statistic (Newey-West), and PT test statistics after transactions
X_after = sm.add_constant(data['Daily_Return_After'].shift(1))
model_after = sm.OLS(data['Daily_Return_After'], X_after, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 1})
nw_standard_error_after = np.sqrt(model_after.cov_params().iloc[1, 1])
t_stat_after = model_after.tvalues[1]
pt_stat_after = sm.stats.acorr_ljungbox(model_after.resid, lags=[1], return_df=True)['lb_stat'][1]

# Display results
print("Before Transactions:")
print("Newey-West Standard Error:", nw_standard_error_before)
print("t-Statistic (Newey-West):", t_stat_before)
print("PT Test Statistic:", pt_stat_before)

print("\nAfter Transactions (best threshold):")
print("Newey-West Standard Error:", nw_standard_error_after)
print("t-Statistic (Newey-West):", t_stat_after)
print("PT Test Statistic:", pt_stat_after)


Before Transactions:
Newey-West Standard Error: 0.06476921639475404
t-Statistic (Newey-West): -1.5744859200972152
PT Test Statistic: 0.008438907737129856

After Transactions (best threshold):
Newey-West Standard Error: 0.11018346478496442
t-Statistic (Newey-West): -0.9816628084017852
PT Test Statistic: 0.005223144317371373


In [19]:
import pandas as pd

# Create a DataFrame for tabular output
results_table1 = pd.DataFrame({
    'Analysis': ['VaR (1%)', 'CVaR (1%)', 'VaR (5%)', 'CVaR (5%)', 'Mean Return (Short)', 'Mean Return (Long)',
                 'Median', 'Quartile 1', 'Quartile 3', 'Newey-West Std. Error', 't-Statistic (Newey-West)',
                 'PT Test Statistic', 'Skewness', 'Kurtosis', 'Max Drawdown', 'Calmar Ratio'],
    'Before Transaction': [var_1_percent_before, cvar_1_percent_before, var_5_percent_before, cvar_5_percent_before,
                           mean_return_short_before, mean_return_long_before, sorted_returns_before.median(),
                           sorted_returns_before.quantile(0.25), sorted_returns_before.quantile(0.75),
                           nw_standard_error_before, t_stat_before, pt_stat_before, skewness_before, kurtosis_before,
                           max_drawdown_before, calmar_ratio_before],
    'After Transaction': [var_1_percent_after, cvar_1_percent_after, var_5_percent_after, cvar_5_percent_after,
                          mean_return_short_after, mean_return_long_after, sorted_returns_after.median(),
                          sorted_returns_after.quantile(0.25), sorted_returns_after.quantile(0.75),
                          nw_standard_error_after, t_stat_after, pt_stat_after, skewness_after, kurtosis_after,
                          max_drawdown_after, calmar_ratio_after]
})

# Round off the numbers to 5 digits
results_table1['Before Transaction'] = results_table1['Before Transaction'].round(5)
results_table1['After Transaction'] = results_table1['After Transaction'].round(5)

# Display the tabular results
print(results_table1)


                    Analysis  Before Transaction  After Transaction
0                   VaR (1%)            -0.00987           -0.00931
1                  CVaR (1%)            -0.01290           -0.01186
2                   VaR (5%)            -0.00535           -0.00265
3                  CVaR (5%)            -0.00833           -0.00638
4        Mean Return (Short)            -0.00252           -0.00392
5         Mean Return (Long)             0.00215            0.00053
6                     Median             0.00016            0.00000
7                 Quartile 1            -0.00136            0.00000
8                 Quartile 3             0.00167            0.00000
9      Newey-West Std. Error             0.06477            0.11018
10  t-Statistic (Newey-West)            -1.57449           -0.98166
11         PT Test Statistic             0.00844            0.00522
12                  Skewness            -0.68981           -1.30913
13                  Kurtosis             3.43205

# We'll define two rules: one for buying (taking long positions) and one for selling (taking short positions). The buying rule will be triggered when the predicted next closing price is significantly higher than the current closing price, indicating an expected price increase. The selling rule will be triggered when the predicted next closing price is significantly lower than the current closing price, indicating an expected price decrease.

## Random Forest

In [20]:
data = df.copy()
data.reset_index(inplace=True)
data['Next_Close'] = data['IGIB_CLOSE'].shift(-1)
data['Day'] = data['DATE'].dt.day
data['Month'] = data['DATE'].dt.month
data['Year'] = data['DATE'].dt.year
#print(data.head())
#data = data[['Day', 'Month', 'Year', 'Next_Close','Hour','Minute']]
data.dropna(inplace=True)

X = data[['Day', 'Month', 'Year']]
y = data['Next_Close']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardize features for each feature to a mean of 0 and a standard deviation of 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Train Random Forest Regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=0)
rf_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_rf = rf_model.predict(X_test_scaled)

# Display metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print("Random Forest Mean Squared Error:", mse_rf)
print("Random Forest R-squared:", r2_rf)

# Now, you can use y_pred_rf for further analysis or visualization.


Random Forest Mean Squared Error: 0.0024488622692489938
Random Forest R-squared: 0.9991694007791434


In [21]:
# Convert y_pred_rf to a DataFrame and set the index to match X_test
y_pred_df1 = pd.DataFrame(y_pred_rf, index=X_test.index, columns=['a_PNClose'])

In [22]:
# Join/merge the predicted values with the original dataset
data2 = data.join(y_pred_df1, how='left')

In [23]:
# Now, data_with_predictions_clean will only contain rows that have a predicted closing value
print(y_pred_df1)
print(data2.head())

        a_PNClose
134009  59.185428
145591  58.105453
88355   59.734371
140459  59.330790
100870  60.637050
...           ...
95341   60.234236
12084   59.050344
162004  54.859840
112624  60.870518
141822  58.547831

[32871 rows x 1 columns]
                 DATE  IGIB_VOLUME  IGIB_OPEN  IGIB_HIGHT  IGIB_LOW  \
0 2020-04-07 17:33:00        492.0    55.2935     55.2935     55.29   
1 2020-04-07 17:34:00       5939.0    55.3050     55.3050     55.29   
2 2020-04-07 17:35:00       6469.0    55.2915     55.3000     55.28   
3 2020-04-07 17:36:00       3131.0    55.2900     55.2900     55.28   
4 2020-04-07 17:37:00       1023.0    55.2900     55.3000     55.29   

   IGIB_CLOSE  IGIB_VW  IGIB_TRANSACTIONS  Next_Close  Day  Month  Year  \
0     55.2900  55.2914                  4     55.2902    7      4  2020   
1     55.2902  55.2968                 40     55.2850    7      4  2020   
2     55.2850  55.2896                 66     55.2900    7      4  2020   
3     55.2900  55.2881         

In [24]:
tdata_resampled = data2.copy()
tdata_resampled.columns = [c.split('_')[-1] for c in tdata_resampled.columns]
tdata_resampled = tdata_resampled.reset_index()

# Resample the data to daily frequency and aggregate
tdata_resampled = tdata_resampled.set_index('DATE').resample('D').agg({
    'OPEN': 'first',
    'HIGHT': 'max',
    'LOW': 'min',
    'CLOSE': 'last',
    'VOLUME': 'sum',
    'VW': 'mean',
    'TRANSACTIONS': 'sum',
    'PNClose': 'last'
})

# Drop NaN values and reset the index
tdata_resampled = tdata_resampled.dropna().reset_index()

print(tdata_resampled.head())

        DATE     OPEN    HIGHT    LOW    CLOSE      VOLUME         VW  \
0 2020-04-07  55.2935  55.5000  55.21  55.2500   1181834.0  55.290144   
1 2020-04-08  55.3500  55.9700  55.33  55.8600   1450054.0  55.722456   
2 2020-04-09  56.4600  57.5538  53.82  57.5538  24143297.0  56.760171   
3 2020-04-13  57.2600  57.5500  56.55  57.3401   4186936.0  57.307612   
4 2020-04-14  57.8200  57.8400  57.26  57.3100   2357930.0  57.516740   

   TRANSACTIONS    PNClose  
0          2863  55.293340  
1          4890  55.725193  
2         52768  56.763498  
3         15114  57.317435  
4         11466  57.516208  


In [25]:
strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
strategy, type(strategy)

(Strategy: Simple Strategy, strategy.Strategy)

In [26]:
rfdata = tdata_resampled.copy()
# Define the threshold for considering a significant price change
price_change_threshold = 0.005  # 1% change, you can adjust this based on your strategy
# Preprocess data
rfdata['Buy_Signal'] = (rfdata['PNClose'] / rfdata['CLOSE']) - 1 > price_change_threshold
rfdata['Sell_Signal'] = (rfdata['PNClose'] / rfdata['CLOSE']) - 1 < -price_change_threshold


In [27]:
strategy.load_data(rfdata, date_column='DATE')
strategy.data.head()

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS,PNClose,Buy_Signal,Sell_Signal
0,2020-04-07,55.2935,55.5000,55.21,55.2500,1181834.0,55.290144,2863,55.293340,False,False
1,2020-04-08,55.3500,55.9700,55.33,55.8600,1450054.0,55.722456,4890,55.725193,False,False
2,2020-04-09,56.4600,57.5538,53.82,57.5538,24143297.0,56.760171,52768,56.763498,False,True
3,2020-04-13,57.2600,57.5500,56.55,57.3401,4186936.0,57.307612,15114,57.317435,False,False
4,2020-04-14,57.8200,57.8400,57.26,57.3100,2357930.0,57.516740,11466,57.516208,False,False


In [28]:
true_counts = rfdata[['Buy_Signal']].sum()
print(true_counts)
true_counts2 = rfdata[['Sell_Signal']].sum()
print(true_counts2)

Buy_Signal    2
dtype: int64
Sell_Signal    4
dtype: int64


In [29]:
# Create rules based on preprocessed data
buy_rule = strategy.add_single_rule('Buy_Signal[N-1]==1', action=MarketAction.BUY, action_quantity='ALL')
sell_rule = strategy.add_single_rule('Sell_Signal[N-1]==1', action=MarketAction.SELL, action_quantity='ALL')

In [30]:
strategy.simulate()

In [31]:
strategy.summary()


Strategy Name: Simple Strategy
First Trade Date: 2021-02-25 00:00:00
Last Trade Date: 2022-01-26 00:00:00
Total Trades: 3
Final PnL: 30.684545069264004
Starting Cash: 1000
Final Cash: 0.0007767762917865184
Final Positions: 17.88760444798633
Final Portfolio Value: 1030.684545069264
Total Commission: 2.565222830423836
Total Buy Commission: 1.7298015264115763
Total Sell Commission: 0.8354213040122596
Total Buy Amount: 2031.5774494180462
Total Sell Amount: 1031.578226194338
Total Buy Quantity: 34.59603052823152
Total Sell Quantity: 16.70842608024519
Total Buy Trades: 2
Total Sell Trades: 1


In [32]:

# Define the range of threshold values to test
threshold_values = np.arange(0.001, 0.01, 0.001)  # Adjust the range and step as needed

# Initialize variables to store the best threshold and its corresponding profit
best_threshold = None
max_profit = 0

# Iterate over threshold values
for threshold in threshold_values:
    # Create a new instance of the strategy for each iteration
    strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
    
    # Preprocess data based on the current threshold
    rfdata['Buy_Signal'] = (rfdata['PNClose'] / rfdata['CLOSE']) - 1 > threshold
    rfdata['Sell_Signal'] = (rfdata['PNClose'] / rfdata['CLOSE']) - 1 < -threshold

    strategy.load_data(rfdata, date_column='DATE')

    # Define rules
    strategy.add_single_rule('Buy_Signal[N-1]==1', action=MarketAction.BUY, action_quantity='ALL')
    strategy.add_single_rule('Sell_Signal[N-1]==1', action=MarketAction.SELL, action_quantity='ALL')

    # Simulate the strategy
    strategy.simulate()

    # Get final portfolio value
    # Check if any trades were made
    if not strategy.history_df.empty:
        # Get final portfolio value
        final_portfolio_value = strategy.history_df['portfolio_value'].iloc[-1]
    else:
        # If no trades were made, use the starting cash value as the final portfolio value
        final_portfolio_value = strategy._starting_cash

    # Check if this is the best threshold so far
    if final_portfolio_value > max_profit:
        max_profit = final_portfolio_value
        best_threshold = threshold

    print(f"Threshold: {threshold}, Final Portfolio Value: {final_portfolio_value}")

print(f"Best Threshold: {best_threshold}, Maximum Profit: {max_profit}")


Threshold: 0.001, Final Portfolio Value: 1013.2413387495313
Threshold: 0.002, Final Portfolio Value: 1011.4075138722274
Threshold: 0.003, Final Portfolio Value: 1047.6380291774842
Threshold: 0.004, Final Portfolio Value: 964.6162095227659
Threshold: 0.005, Final Portfolio Value: 1030.684545069264
Threshold: 0.006, Final Portfolio Value: 1000
Threshold: 0.007, Final Portfolio Value: 1000
Threshold: 0.008, Final Portfolio Value: 1000
Threshold: 0.009000000000000001, Final Portfolio Value: 1000
Best Threshold: 0.003, Maximum Profit: 1047.6380291774842


In [33]:
import numpy as np

# Find the best threshold for after transactions
best_threshold_after = 0.003  # Replace with the actual best threshold you found

# Apply the best threshold to create binary columns for significant price changes before and after transactions
rfdata['Significant_Price_Change_Before'] = (rfdata['PNClose'] / rfdata['CLOSE'].shift(1)) - 1 > best_threshold_after
rfdata['Significant_Price_Change_After'] = (rfdata['PNClose'] / rfdata['CLOSE']) - 1 > best_threshold_after

# Calculate daily returns before and after transactions based on the best threshold
rfdata['Daily_Return_Before'] = rfdata['CLOSE'].pct_change()
rfdata['Daily_Return_After'] = rfdata['CLOSE'].pct_change() * rfdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
rfdata.dropna(inplace=True)

# Sort daily returns before and after transactions based on the best threshold
sorted_returns_before = rfdata['Daily_Return_Before'].sort_values()
sorted_returns_after = rfdata['Daily_Return_After'].sort_values()

# Calculate the index for the VaR and CVaR before transactions based on the best threshold
index_1_percent_before = int(0.01 * len(sorted_returns_before))
index_5_percent_before = int(0.05 * len(sorted_returns_before))

# Calculate VaR and CVaR before transactions based on the best threshold
var_1_percent_before = sorted_returns_before.iloc[index_1_percent_before]
cvar_1_percent_before = sorted_returns_before.iloc[:index_1_percent_before].mean()

var_5_percent_before = sorted_returns_before.iloc[index_5_percent_before]
cvar_5_percent_before = sorted_returns_before.iloc[:index_5_percent_before].mean()

# Calculate the index for the VaR and CVaR after transactions based on the best threshold
index_1_percent_after = int(0.01 * len(sorted_returns_after))
index_5_percent_after = int(0.05 * len(sorted_returns_after))

# Calculate VaR and CVaR after transactions based on the best threshold
var_1_percent_after = sorted_returns_after.iloc[index_1_percent_after]
cvar_1_percent_after = sorted_returns_after.iloc[:index_1_percent_after].mean()

var_5_percent_after = sorted_returns_after.iloc[index_5_percent_after]
cvar_5_percent_after = sorted_returns_after.iloc[:index_5_percent_after].mean()

# Display results
print("VaR (1%) before transactions (best threshold):", var_1_percent_before)
print("CVaR (1%) before transactions (best threshold):", cvar_1_percent_before)

print("VaR (5%) before transactions (best threshold):", var_5_percent_before)
print("CVaR (5%) before transactions (best threshold):", cvar_5_percent_before)

print("VaR (1%) after transactions (best threshold):", var_1_percent_after)
print("CVaR (1%) after transactions (best threshold):", cvar_1_percent_after)

print("VaR (5%) after transactions (best threshold):", var_5_percent_after)
print("CVaR (5%) after transactions (best threshold):", cvar_5_percent_after)


VaR (1%) before transactions (best threshold): -0.009872147596698522
CVaR (1%) before transactions (best threshold): -0.012902696338238307
VaR (5%) before transactions (best threshold): -0.005346700083542211
CVaR (5%) before transactions (best threshold): -0.008327159436293878
VaR (1%) after transactions (best threshold): -0.006003001500750393
CVaR (1%) after transactions (best threshold): -0.011964517170051248
VaR (5%) after transactions (best threshold): -0.0
CVaR (5%) after transactions (best threshold): -0.003079914859835395


In [34]:
# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.003  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
rfdata['Significant_Price_Change_After'] = (rfdata['PNClose'] / rfdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
rfdata['Daily_Return_Before'] = rfdata['CLOSE'].pct_change()
rfdata['Daily_Return_After'] = rfdata['CLOSE'].pct_change() * rfdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
rfdata.dropna(inplace=True)

# Calculate mean return (short) before transactions
mean_return_short_before = rfdata[rfdata['Daily_Return_Before'] < 0]['Daily_Return_Before'].mean()

# Calculate mean return (long) before transactions
mean_return_long_before = rfdata[rfdata['Daily_Return_Before'] >= 0]['Daily_Return_Before'].mean()

# Calculate mean return (short) after transactions
mean_return_short_after = rfdata[rfdata['Daily_Return_After'] < 0]['Daily_Return_After'].mean()

# Calculate mean return (long) after transactions
mean_return_long_after = rfdata[rfdata['Daily_Return_After'] >= 0]['Daily_Return_After'].mean()

# Display results
print("Mean Return (Short) before transactions:", mean_return_short_before)
print("Mean Return (Long) before transactions:", mean_return_long_before)

print("Mean Return (Short) after transactions (best threshold):", mean_return_short_after)
print("Mean Return (Long) after transactions (best threshold):", mean_return_long_after)


Mean Return (Short) before transactions: -0.0025218856950098237
Mean Return (Long) before transactions: 0.0021452901037482684
Mean Return (Short) after transactions (best threshold): -0.0005249380451027852
Mean Return (Long) after transactions (best threshold): 0.0004184544941156642


In [35]:
import numpy as np

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.003  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
rfdata['Significant_Price_Change_After'] = (rfdata['PNClose'] / rfdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
rfdata['Daily_Return_Before'] = rfdata['CLOSE'].pct_change()
rfdata['Daily_Return_After'] = rfdata['CLOSE'].pct_change() * rfdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
rfdata.dropna(inplace=True)

# Calculate statistics before transactions
stats_before = rfdata['Daily_Return_Before'].describe()

# Calculate statistics after transactions
stats_after = rfdata['Daily_Return_After'].describe()

# Display results
print("Statistics before transactions:")
print("Minimum:", stats_before['min'])
print("Quartile 1:", stats_before['25%'])
print("Median:", stats_before['50%'])
print("Quartile 3:", stats_before['75%'])
print("Maximum:", stats_before['max'])

print("\nStatistics after transactions (best threshold):")
print("Minimum:", stats_after['min'])
print("Quartile 1:", stats_after['25%'])
print("Median:", stats_after['50%'])
print("Quartile 3:", stats_after['75%'])
print("Maximum:", stats_after['max'])


Statistics before transactions:
Minimum: -0.016727272727272813
Quartile 1: -0.0013793745422527315
Median: 0.00016258849448158585
Quartile 3: 0.0016637018037399254
Maximum: 0.010556300268096619

Statistics after transactions (best threshold):
Minimum: -0.0005249380451027852
Quartile 1: 0.0
Median: -0.0
Quartile 3: 0.0
Maximum: 0.010556300268096619


In [36]:
# Apply the best threshold to create a binary column for significant price changes after transactions
rfdata['Significant_Price_Change_After'] = (rfdata['PNClose'] / rfdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
rfdata['Daily_Return_Before'] = rfdata['CLOSE'].pct_change()
rfdata['Daily_Return_After'] = rfdata['CLOSE'].pct_change() * rfdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
rfdata.dropna(inplace=True)

# Calculate standard deviation before transactions
std_dev_before = rfdata['Daily_Return_Before'].std()

# Calculate standard deviation after transactions based on the best threshold
std_dev_after = rfdata['Daily_Return_After'].std()

# Display results
print("Standard Deviation before transactions:", std_dev_before)
print("Standard Deviation after transactions (best threshold):", std_dev_after)


Standard Deviation before transactions: 0.0031907795962179742
Standard Deviation after transactions (best threshold): 0.0014237961244899465


In [37]:
import numpy as np

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.003  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
rfdata['Significant_Price_Change_After'] = (rfdata['PNClose'] / rfdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
rfdata['Daily_Return_Before'] = rfdata['CLOSE'].pct_change()
rfdata['Daily_Return_After'] = rfdata['CLOSE'].pct_change() * rfdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
rfdata.dropna(inplace=True)

# Skewness before transactions
skewness_before = rfdata['Daily_Return_Before'].skew()

# Kurtosis before transactions
kurtosis_before = rfdata['Daily_Return_Before'].kurtosis()

# Calculate drawdowns before transactions
cumulative_returns_before = (1 + rfdata['Daily_Return_Before']).cumprod()
running_max_before = cumulative_returns_before.cummax()
drawdown_before = (cumulative_returns_before / running_max_before) - 1
max_drawdown_before = drawdown_before.min()

# Calmar ratio before transactions
calmar_ratio_before = cumulative_returns_before.iloc[-1] / abs(max_drawdown_before)

# Skewness after transactions
skewness_after = rfdata['Daily_Return_After'].skew()

# Kurtosis after transactions
kurtosis_after = rfdata['Daily_Return_After'].kurtosis()

# Calculate drawdowns after transactions
cumulative_returns_after = (1 + rfdata['Daily_Return_After']).cumprod()
running_max_after = cumulative_returns_after.cummax()
drawdown_after = (cumulative_returns_after / running_max_after) - 1
max_drawdown_after = drawdown_after.min()

# Calmar ratio after transactions
calmar_ratio_after = cumulative_returns_after.iloc[-1] / abs(max_drawdown_after)

# Display results
print("Skewness before transactions:", skewness_before)
print("Kurtosis before transactions:", kurtosis_before)
print("Maximum Drawdown before transactions:", max_drawdown_before)
print("Calmar Ratio before transactions:", calmar_ratio_before)

print("\nSkewness after transactions (best threshold):", skewness_after)
print("Kurtosis after transactions (best threshold):", kurtosis_after)
print("Maximum Drawdown after transactions (best threshold):", max_drawdown_after)
print("Calmar Ratio after transactions (best threshold):", calmar_ratio_after)


Skewness before transactions: -0.6898074585317826
Kurtosis before transactions: 3.4320522799877202
Maximum Drawdown before transactions: -0.12491909385113098
Calmar Ratio before transactions: 7.60708735239591

Skewness after transactions (best threshold): 4.380354860913198
Kurtosis after transactions (best threshold): 20.140093513604082
Maximum Drawdown after transactions (best threshold): 0.0
Calmar Ratio after transactions (best threshold): inf


In [38]:
import statsmodels.api as sm

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.003  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
rfdata['Significant_Price_Change_After'] = (rfdata['PNClose'] / rfdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
rfdata['Daily_Return_Before'] = rfdata['CLOSE'].pct_change()
rfdata['Daily_Return_After'] = rfdata['CLOSE'].pct_change() * rfdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
rfdata.dropna(inplace=True)

# Calculate Newey-West standard error, t-statistic (Newey-West), and PT test statistics before transactions
X_before = sm.add_constant(rfdata['Daily_Return_Before'].shift(1))
model_before = sm.OLS(rfdata['Daily_Return_Before'], X_before, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 1})
nw_standard_error_before = np.sqrt(model_before.cov_params().iloc[1, 1])
t_stat_before = model_before.tvalues[1]
pt_stat_before = sm.stats.acorr_ljungbox(model_before.resid, lags=[1], return_df=True)['lb_stat'][1]

# Calculate Newey-West standard error, t-statistic (Newey-West), and PT test statistics after transactions
X_after = sm.add_constant(rfdata['Daily_Return_After'].shift(1))
model_after = sm.OLS(rfdata['Daily_Return_After'], X_after, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 1})
nw_standard_error_after = np.sqrt(model_after.cov_params().iloc[1, 1])
t_stat_after = model_after.tvalues[1]
pt_stat_after = sm.stats.acorr_ljungbox(model_after.resid, lags=[1], return_df=True)['lb_stat'][1]

# Display results
print("Before Transactions:")
print("Newey-West Standard Error:", nw_standard_error_before)
print("t-Statistic (Newey-West):", t_stat_before)
print("PT Test Statistic:", pt_stat_before)

print("\nAfter Transactions (best threshold):")
print("Newey-West Standard Error:", nw_standard_error_after)
print("t-Statistic (Newey-West):", t_stat_after)
print("PT Test Statistic:", pt_stat_after)


Before Transactions:
Newey-West Standard Error: 0.06476921639475404
t-Statistic (Newey-West): -1.5744859200972152
PT Test Statistic: 0.008438907737129856

After Transactions (best threshold):
Newey-West Standard Error: 0.045221813563508934
t-Statistic (Newey-West): 0.4412055643953053
PT Test Statistic: 0.0012458864415944265


In [39]:
import pandas as pd

# Create a DataFrame for tabular output
results_table2 = pd.DataFrame({
    'Analysis': ['VaR (1%)', 'CVaR (1%)', 'VaR (5%)', 'CVaR (5%)', 'Mean Return (Short)', 'Mean Return (Long)',
                 'Median', 'Quartile 1', 'Quartile 3', 'Newey-West Std. Error', 't-Statistic (Newey-West)',
                 'PT Test Statistic', 'Skewness', 'Kurtosis', 'Max Drawdown', 'Calmar Ratio'],
    'Before Transaction': [var_1_percent_before, cvar_1_percent_before, var_5_percent_before, cvar_5_percent_before,
                           mean_return_short_before, mean_return_long_before, sorted_returns_before.median(),
                           sorted_returns_before.quantile(0.25), sorted_returns_before.quantile(0.75),
                           nw_standard_error_before, t_stat_before, pt_stat_before, skewness_before, kurtosis_before,
                           max_drawdown_before, calmar_ratio_before],
    'After Transaction': [var_1_percent_after, cvar_1_percent_after, var_5_percent_after, cvar_5_percent_after,
                          mean_return_short_after, mean_return_long_after, sorted_returns_after.median(),
                          sorted_returns_after.quantile(0.25), sorted_returns_after.quantile(0.75),
                          nw_standard_error_after, t_stat_after, pt_stat_after, skewness_after, kurtosis_after,
                          max_drawdown_after, calmar_ratio_after]
})

# Round off the numbers to 5 digits
results_table2['Before Transaction'] = results_table2['Before Transaction'].round(5)
results_table2['After Transaction'] = results_table2['After Transaction'].round(5)

# Display the tabular results
print(results_table2)


                    Analysis  Before Transaction  After Transaction
0                   VaR (1%)            -0.00987           -0.00600
1                  CVaR (1%)            -0.01290           -0.01196
2                   VaR (5%)            -0.00535           -0.00000
3                  CVaR (5%)            -0.00833           -0.00308
4        Mean Return (Short)            -0.00252           -0.00052
5         Mean Return (Long)             0.00215            0.00042
6                     Median             0.00016           -0.00000
7                 Quartile 1            -0.00136            0.00000
8                 Quartile 3             0.00167            0.00000
9      Newey-West Std. Error             0.06477            0.04522
10  t-Statistic (Newey-West)            -1.57449            0.44121
11         PT Test Statistic             0.00844            0.00125
12                  Skewness            -0.68981            4.38035
13                  Kurtosis             3.43205

## Gradient Boosted

In [40]:
data = df.copy()
data.reset_index(inplace=True)
data['Next_Close'] = data['IGIB_CLOSE'].shift(-1)
data['Day'] = data['DATE'].dt.day
data['Month'] = data['DATE'].dt.month
data['Year'] = data['DATE'].dt.year
#print(data.head())
#data = data[['Day', 'Month', 'Year', 'Next_Close','Hour','Minute']]
data.dropna(inplace=True)

X = data[['Day', 'Month', 'Year']]
y = data['Next_Close']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Standardize features for each feature to a mean of 0 and a standard deviation of 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Gradient Boosted Regression model
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=0)
gb_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred_gb = gb_model.predict(X_test_scaled)

# Display metrics
mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)
print("Gradient Boosted Regression Mean Squared Error:", mse_gb)
print("Gradient Boosted Regression R-squared:", r2_gb)

# Now, you can use y_pred_gb for further analysis or visualization.


Gradient Boosted Regression Mean Squared Error: 0.04215383739391052
Gradient Boosted Regression R-squared: 0.9857023627113848


In [41]:
# Convert y_pred to a DataFrame and set the index to match X_test
y_pred_df1 = pd.DataFrame(y_pred_gb, index=X_test.index, columns=['a_PNClose'])

In [42]:
# Join/merge the predicted values with the original dataset
data3 = data.join(y_pred_df1, how='left')

In [43]:
# Now, data_with_predictions_clean will only contain rows that have a predicted closing value
print(y_pred_df1)
print(data3.head())

        a_PNClose
134009  59.476671
145591  57.898570
88355   59.735843
140459  59.447510
100870  60.645732
...           ...
95341   60.325231
12084   59.249780
162004  54.731901
112624  60.771428
141822  58.593538

[32871 rows x 1 columns]
                 DATE  IGIB_VOLUME  IGIB_OPEN  IGIB_HIGHT  IGIB_LOW  \
0 2020-04-07 17:33:00        492.0    55.2935     55.2935     55.29   
1 2020-04-07 17:34:00       5939.0    55.3050     55.3050     55.29   
2 2020-04-07 17:35:00       6469.0    55.2915     55.3000     55.28   
3 2020-04-07 17:36:00       3131.0    55.2900     55.2900     55.28   
4 2020-04-07 17:37:00       1023.0    55.2900     55.3000     55.29   

   IGIB_CLOSE  IGIB_VW  IGIB_TRANSACTIONS  Next_Close  Day  Month  Year  \
0     55.2900  55.2914                  4     55.2902    7      4  2020   
1     55.2902  55.2968                 40     55.2850    7      4  2020   
2     55.2850  55.2896                 66     55.2900    7      4  2020   
3     55.2900  55.2881         

In [44]:
tdata_resampled1 = data3.copy()
tdata_resampled1.columns = [c.split('_')[-1] for c in tdata_resampled1.columns]
tdata_resampled1 = tdata_resampled1.reset_index()

# Resample the data to daily frequency and aggregate
tdata_resampled1 = tdata_resampled1.set_index('DATE').resample('D').agg({
    'OPEN': 'first',
    'HIGHT': 'max',
    'LOW': 'min',
    'CLOSE': 'last',
    'VOLUME': 'sum',
    'VW': 'mean',
    'TRANSACTIONS': 'sum',
    'PNClose': 'last'
})

# Drop NaN values and reset the index
tdata_resampled1 = tdata_resampled1.dropna().reset_index()

print(tdata_resampled1.head())

        DATE     OPEN    HIGHT    LOW    CLOSE      VOLUME         VW  \
0 2020-04-07  55.2935  55.5000  55.21  55.2500   1181834.0  55.290144   
1 2020-04-08  55.3500  55.9700  55.33  55.8600   1450054.0  55.722456   
2 2020-04-09  56.4600  57.5538  53.82  57.5538  24143297.0  56.760171   
3 2020-04-13  57.2600  57.5500  56.55  57.3401   4186936.0  57.307612   
4 2020-04-14  57.8200  57.8400  57.26  57.3100   2357930.0  57.516740   

   TRANSACTIONS    PNClose  
0          2863  56.434629  
1          4890  56.434629  
2         52768  57.083366  
3         15114  57.130678  
4         11466  57.246674  


In [45]:
strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
strategy, type(strategy)

(Strategy: Simple Strategy, strategy.Strategy)

In [46]:
gbdata = tdata_resampled1.copy()
# Define the threshold for considering a significant price change
price_change_threshold = 0.005  # 1% change, you can adjust this based on your strategy
# Preprocess data
gbdata['Buy_Signal'] = (gbdata['PNClose'] / gbdata['CLOSE']) - 1 > price_change_threshold
gbdata['Sell_Signal'] = (gbdata['PNClose'] / gbdata['CLOSE']) - 1 < -price_change_threshold


In [47]:
strategy.load_data(gbdata, date_column='DATE')
strategy.data.head()

,DATE,OPEN,HIGHT,LOW,CLOSE,VOLUME,VW,TRANSACTIONS,PNClose,Buy_Signal,Sell_Signal
0,2020-04-07,55.2935,55.5000,55.21,55.2500,1181834.0,55.290144,2863,56.434629,True,False
1,2020-04-08,55.3500,55.9700,55.33,55.8600,1450054.0,55.722456,4890,56.434629,True,False
2,2020-04-09,56.4600,57.5538,53.82,57.5538,24143297.0,56.760171,52768,57.083366,False,True
3,2020-04-13,57.2600,57.5500,56.55,57.3401,4186936.0,57.307612,15114,57.130678,False,False
4,2020-04-14,57.8200,57.8400,57.26,57.3100,2357930.0,57.516740,11466,57.246674,False,False


In [48]:
true_counts = gbdata[['Buy_Signal']].sum()
print(true_counts)
true_counts2 = gbdata[['Sell_Signal']].sum()
print(true_counts2)

Buy_Signal    38
dtype: int64
Sell_Signal    42
dtype: int64


In [49]:
# Create rules based on preprocessed data
buy_rule = strategy.add_single_rule('Buy_Signal[N-1]==1', action=MarketAction.BUY, action_quantity='ALL')
sell_rule = strategy.add_single_rule('Sell_Signal[N-1]==1', action=MarketAction.SELL, action_quantity='ALL')

In [50]:
strategy.simulate()

In [51]:
strategy.summary()

Strategy Name: Simple Strategy
First Trade Date: 2020-04-07 00:00:00
Last Trade Date: 2022-03-25 00:00:00
Total Trades: 21
Final PnL: 93.39320908541822
Starting Cash: 1000
Final Cash: 0.0009282746482313087
Final Positions: 20.13984676387493
Final Portfolio Value: 1093.3932090854182
Total Commission: 19.49440931483169
Total Buy Commission: 10.25070082651272
Total Sell Commission: 9.243708488318973
Total Buy Amount: 11935.954165981453
Total Sell Amount: 10935.955094256104
Total Buy Quantity: 205.01401653025437
Total Sell Quantity: 184.87416976637942
Total Buy Trades: 11
Total Sell Trades: 10


In [52]:
# Define the range of threshold values to test
threshold_values = np.arange(0.001, 0.01, 0.001)  # Adjust the range and step as needed

# Initialize variables to store the best threshold and its corresponding profit
best_threshold = None
max_profit = 0

# Iterate over threshold values
for threshold in threshold_values:
    # Create a new instance of the strategy for each iteration
    strategy = Strategy(name="Simple Strategy", cash=1000, commission=0.05, min_positions=1)
    
    # Preprocess data based on the current threshold
    gbdata['Buy_Signal'] = (gbdata['PNClose'] / gbdata['CLOSE']) - 1 > threshold
    gbdata['Sell_Signal'] = (gbdata['PNClose'] / gbdata['CLOSE']) - 1 < -threshold

    strategy.load_data(gbdata, date_column='DATE')

    # Define rules
    strategy.add_single_rule('Buy_Signal[N-1]==1', action=MarketAction.BUY, action_quantity='ALL')
    strategy.add_single_rule('Sell_Signal[N-1]==1', action=MarketAction.SELL, action_quantity='ALL')

    # Simulate the strategy
    strategy.simulate()

    # Get final portfolio value
    # Check if any trades were made
    if not strategy.history_df.empty:
        # Get final portfolio value
        final_portfolio_value = strategy.history_df['portfolio_value'].iloc[-1]
    else:
        # If no trades were made, use the starting cash value as the final portfolio value
        final_portfolio_value = strategy._starting_cash

    # Check if this is the best threshold so far
    if final_portfolio_value > max_profit:
        max_profit = final_portfolio_value
        best_threshold = threshold

    print(f"Threshold: {threshold}, Final Portfolio Value: {final_portfolio_value}")

print(f"Best Threshold: {best_threshold}, Maximum Profit: {max_profit}")


Threshold: 0.001, Final Portfolio Value: 1143.1685970979786
Threshold: 0.002, Final Portfolio Value: 1133.8474760259376
Threshold: 0.003, Final Portfolio Value: 1140.2953106566072
Threshold: 0.004, Final Portfolio Value: 1113.7064997827522
Threshold: 0.005, Final Portfolio Value: 1093.3932090854182
Threshold: 0.006, Final Portfolio Value: 1090.8478685433547
Threshold: 0.007, Final Portfolio Value: 1052.6207325826615
Threshold: 0.008, Final Portfolio Value: 1039.1087989405094
Threshold: 0.009000000000000001, Final Portfolio Value: 1090.1526162376263
Best Threshold: 0.001, Maximum Profit: 1143.1685970979786


In [53]:
# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
gbdata['Significant_Price_Change_After'] = (gbdata['PNClose'] / gbdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
gbdata['Daily_Return_Before'] = gbdata['CLOSE'].pct_change()
gbdata['Daily_Return_After'] = gbdata['CLOSE'].pct_change() * gbdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
gbdata.dropna(inplace=True)

# Sort daily returns before and after transactions based on the best threshold
sorted_returns_before = gbdata['Daily_Return_Before'].sort_values()
sorted_returns_after = gbdata['Daily_Return_After'].sort_values()

# Calculate the index for the VaR and CVaR before transactions based on the best threshold
index_1_percent_before = int(0.01 * len(sorted_returns_before))
index_5_percent_before = int(0.05 * len(sorted_returns_before))

# Calculate VaR and CVaR before transactions based on the best threshold
var_1_percent_before = sorted_returns_before.iloc[index_1_percent_before]
cvar_1_percent_before = sorted_returns_before.iloc[:index_1_percent_before].mean()

var_5_percent_before = sorted_returns_before.iloc[index_5_percent_before]
cvar_5_percent_before = sorted_returns_before.iloc[:index_5_percent_before].mean()

# Calculate the index for the VaR and CVaR after transactions based on the best threshold
index_1_percent_after = int(0.01 * len(sorted_returns_after))
index_5_percent_after = int(0.05 * len(sorted_returns_after))

# Calculate VaR and CVaR after transactions based on the best threshold
var_1_percent_after = sorted_returns_after.iloc[index_1_percent_after]
cvar_1_percent_after = sorted_returns_after.iloc[:index_1_percent_after].mean()

var_5_percent_after = sorted_returns_after.iloc[index_5_percent_after]
cvar_5_percent_after = sorted_returns_after.iloc[:index_5_percent_after].mean()

# Display results
print("VaR (1%) before transactions (best threshold):", var_1_percent_before)
print("CVaR (1%) before transactions (best threshold):", cvar_1_percent_before)

print("VaR (5%) before transactions (best threshold):", var_5_percent_before)
print("CVaR (5%) before transactions (best threshold):", cvar_5_percent_before)

print("VaR (1%) after transactions (best threshold):", var_1_percent_after)
print("CVaR (1%) after transactions (best threshold):", cvar_1_percent_after)

print("VaR (5%) after transactions (best threshold):", var_5_percent_after)
print("CVaR (5%) after transactions (best threshold):", cvar_5_percent_after)


VaR (1%) before transactions (best threshold): -0.009872147596698522
CVaR (1%) before transactions (best threshold): -0.012902696338238307
VaR (5%) before transactions (best threshold): -0.005346700083542211
CVaR (5%) before transactions (best threshold): -0.008327159436293878
VaR (1%) after transactions (best threshold): -0.0031708945260348065
CVaR (1%) after transactions (best threshold): -0.004276450229820594
VaR (5%) after transactions (best threshold): -0.0013151405556468543
CVaR (5%) after transactions (best threshold): -0.0027415006065125034


In [54]:
# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
gbdata['Significant_Price_Change_After'] = (gbdata['PNClose'] / gbdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
gbdata['Daily_Return_Before'] = gbdata['CLOSE'].pct_change()
gbdata['Daily_Return_After'] = gbdata['CLOSE'].pct_change() * gbdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
gbdata.dropna(inplace=True)

# Calculate mean return (short) before transactions
mean_return_short_before = gbdata[gbdata['Daily_Return_Before'] < 0]['Daily_Return_Before'].mean()

# Calculate mean return (long) before transactions
mean_return_long_before = gbdata[gbdata['Daily_Return_Before'] >= 0]['Daily_Return_Before'].mean()

# Calculate mean return (short) after transactions
mean_return_short_after = gbdata[gbdata['Daily_Return_After'] < 0]['Daily_Return_After'].mean()

# Calculate mean return (long) after transactions
mean_return_long_after = gbdata[gbdata['Daily_Return_After'] >= 0]['Daily_Return_After'].mean()

# Display results
print("Mean Return (Short) before transactions:", mean_return_short_before)
print("Mean Return (Long) before transactions:", mean_return_long_before)

print("Mean Return (Short) after transactions (best threshold):", mean_return_short_after)
print("Mean Return (Long) after transactions (best threshold):", mean_return_long_after)


Mean Return (Short) before transactions: -0.0025218856950098237
Mean Return (Long) before transactions: 0.0021452901037482684
Mean Return (Short) after transactions (best threshold): -0.0017467303916084908
Mean Return (Long) after transactions (best threshold): 0.000778336728045709


In [55]:
import numpy as np

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
gbdata['Significant_Price_Change_After'] = (gbdata['PNClose'] / gbdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
gbdata['Daily_Return_Before'] = gbdata['CLOSE'].pct_change()
gbdata['Daily_Return_After'] = gbdata['CLOSE'].pct_change() * gbdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
gbdata.dropna(inplace=True)

# Calculate statistics before transactions
stats_before = gbdata['Daily_Return_Before'].describe()

# Calculate statistics after transactions
stats_after = gbdata['Daily_Return_After'].describe()

# Display results
print("Statistics before transactions:")
print("Minimum:", stats_before['min'])
print("Quartile 1:", stats_before['25%'])
print("Median:", stats_before['50%'])
print("Quartile 3:", stats_before['75%'])
print("Maximum:", stats_before['max'])

print("\nStatistics after transactions (best threshold):")
print("Minimum:", stats_after['min'])
print("Quartile 1:", stats_after['25%'])
print("Median:", stats_after['50%'])
print("Quartile 3:", stats_after['75%'])
print("Maximum:", stats_after['max'])


Statistics before transactions:
Minimum: -0.016727272727272813
Quartile 1: -0.0013793745422527315
Median: 0.00016258849448158585
Quartile 3: 0.0016637018037399254
Maximum: 0.010556300268096619

Statistics after transactions (best threshold):
Minimum: -0.006013119533527789
Quartile 1: 0.0
Median: -0.0
Quartile 3: 0.00033523823846398626
Maximum: 0.009643592010775803


In [56]:
# Apply the best threshold to create a binary column for significant price changes after transactions
gbdata['Significant_Price_Change_After'] = (gbdata['PNClose'] / gbdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
gbdata['Daily_Return_Before'] = gbdata['CLOSE'].pct_change()
gbdata['Daily_Return_After'] = gbdata['CLOSE'].pct_change() * gbdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
gbdata.dropna(inplace=True)

# Calculate standard deviation before transactions
std_dev_before = gbdata['Daily_Return_Before'].std()

# Calculate standard deviation after transactions based on the best threshold
std_dev_after = gbdata['Daily_Return_After'].std()

# Display results
print("Standard Deviation before transactions:", std_dev_before)
print("Standard Deviation after transactions (best threshold):", std_dev_after)


Standard Deviation before transactions: 0.0031907795962179742
Standard Deviation after transactions (best threshold): 0.0016646134575625203


In [57]:
import numpy as np

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
gbdata['Significant_Price_Change_After'] = (gbdata['PNClose'] / gbdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
gbdata['Daily_Return_Before'] = gbdata['CLOSE'].pct_change()
gbdata['Daily_Return_After'] = gbdata['CLOSE'].pct_change() * gbdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
gbdata.dropna(inplace=True)

# Skewness before transactions
skewness_before = gbdata['Daily_Return_Before'].skew()

# Kurtosis before transactions
kurtosis_before = gbdata['Daily_Return_Before'].kurtosis()

# Calculate drawdowns before transactions
cumulative_returns_before = (1 + gbdata['Daily_Return_Before']).cumprod()
running_max_before = cumulative_returns_before.cummax()
drawdown_before = (cumulative_returns_before / running_max_before) - 1
max_drawdown_before = drawdown_before.min()

# Calmar ratio before transactions
calmar_ratio_before = cumulative_returns_before.iloc[-1] / abs(max_drawdown_before)

# Skewness after transactions
skewness_after = gbdata['Daily_Return_After'].skew()

# Kurtosis after transactions
kurtosis_after = gbdata['Daily_Return_After'].kurtosis()

# Calculate drawdowns after transactions
cumulative_returns_after = (1 + gbdata['Daily_Return_After']).cumprod()
running_max_after = cumulative_returns_after.cummax()
drawdown_after = (cumulative_returns_after / running_max_after) - 1
max_drawdown_after = drawdown_after.min()

# Calmar ratio after transactions
calmar_ratio_after = cumulative_returns_after.iloc[-1] / abs(max_drawdown_after)

# Display results
print("Skewness before transactions:", skewness_before)
print("Kurtosis before transactions:", kurtosis_before)
print("Maximum Drawdown before transactions:", max_drawdown_before)
print("Calmar Ratio before transactions:", calmar_ratio_before)

print("\nSkewness after transactions (best threshold):", skewness_after)
print("Kurtosis after transactions (best threshold):", kurtosis_after)
print("Maximum Drawdown after transactions (best threshold):", max_drawdown_after)
print("Calmar Ratio after transactions (best threshold):", calmar_ratio_after)


Skewness before transactions: -0.6898074585317826
Kurtosis before transactions: 3.4320522799877202
Maximum Drawdown before transactions: -0.12491909385113098
Calmar Ratio before transactions: 7.60708735239591

Skewness after transactions (best threshold): 1.7294701807398674
Kurtosis after transactions (best threshold): 6.860488564450202
Maximum Drawdown after transactions (best threshold): -0.006013119533527789
Calmar Ratio after transactions (best threshold): 210.1861482564115


In [58]:
import statsmodels.api as sm

# Find the threshold that resulted in the maximum return after transactions
best_threshold = 0.001  # Replace with the actual best threshold you found

# Apply the best threshold to create a binary column for significant price changes after transactions
gbdata['Significant_Price_Change_After'] = (gbdata['PNClose'] / gbdata['CLOSE'].shift(1)) - 1 > best_threshold

# Calculate daily returns before and after transactions based on the best threshold
gbdata['Daily_Return_Before'] = gbdata['CLOSE'].pct_change()
gbdata['Daily_Return_After'] = gbdata['CLOSE'].pct_change() * gbdata['Significant_Price_Change_After']

# Drop NaN values created by the percentage change
data.dropna(inplace=True)

# Calculate Newey-West standard error, t-statistic (Newey-West), and PT test statistics before transactions
X_before = sm.add_constant(gbdata['Daily_Return_Before'].shift(1))
model_before = sm.OLS(gbdata['Daily_Return_Before'], X_before, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 1})
nw_standard_error_before = np.sqrt(model_before.cov_params().iloc[1, 1])
t_stat_before = model_before.tvalues[1]
pt_stat_before = sm.stats.acorr_ljungbox(model_before.resid, lags=[1], return_df=True)['lb_stat'][1]

# Calculate Newey-West standard error, t-statistic (Newey-West), and PT test statistics after transactions
X_after = sm.add_constant(gbdata['Daily_Return_After'].shift(1))
model_after = sm.OLS(gbdata['Daily_Return_After'], X_after, missing='drop').fit(cov_type='HAC', cov_kwds={'maxlags': 1})
nw_standard_error_after = np.sqrt(model_after.cov_params().iloc[1, 1])
t_stat_after = model_after.tvalues[1]
pt_stat_after = sm.stats.acorr_ljungbox(model_after.resid, lags=[1], return_df=True)['lb_stat'][1]

# Display results
print("Before Transactions:")
print("Newey-West Standard Error:", nw_standard_error_before)
print("t-Statistic (Newey-West):", t_stat_before)
print("PT Test Statistic:", pt_stat_before)

print("\nAfter Transactions (best threshold):")
print("Newey-West Standard Error:", nw_standard_error_after)
print("t-Statistic (Newey-West):", t_stat_after)
print("PT Test Statistic:", pt_stat_after)


Before Transactions:
Newey-West Standard Error: 0.06476921639475404
t-Statistic (Newey-West): -1.5744859200972152
PT Test Statistic: 0.008438907737129856

After Transactions (best threshold):
Newey-West Standard Error: 0.054466230325154255
t-Statistic (Newey-West): -0.8989062038747693
PT Test Statistic: 0.000308580981178074


In [59]:
import pandas as pd

# Create a DataFrame for tabular output
results_table3 = pd.DataFrame({
    'Analysis': ['VaR (1%)', 'CVaR (1%)', 'VaR (5%)', 'CVaR (5%)', 'Mean Return (Short)', 'Mean Return (Long)',
                 'Median', 'Quartile 1', 'Quartile 3', 'Newey-West Std. Error', 't-Statistic (Newey-West)',
                 'PT Test Statistic', 'Skewness', 'Kurtosis', 'Max Drawdown', 'Calmar Ratio'],
    'Before Transaction': [var_1_percent_before, cvar_1_percent_before, var_5_percent_before, cvar_5_percent_before,
                           mean_return_short_before, mean_return_long_before, sorted_returns_before.median(),
                           sorted_returns_before.quantile(0.25), sorted_returns_before.quantile(0.75),
                           nw_standard_error_before, t_stat_before, pt_stat_before, skewness_before, kurtosis_before,
                           max_drawdown_before, calmar_ratio_before],
    'After Transaction': [var_1_percent_after, cvar_1_percent_after, var_5_percent_after, cvar_5_percent_after,
                          mean_return_short_after, mean_return_long_after, sorted_returns_after.median(),
                          sorted_returns_after.quantile(0.25), sorted_returns_after.quantile(0.75),
                          nw_standard_error_after, t_stat_after, pt_stat_after, skewness_after, kurtosis_after,
                          max_drawdown_after, calmar_ratio_after]
})

# Round off the numbers to 5 digits
results_table3['Before Transaction'] = results_table3['Before Transaction'].round(5)
results_table3['After Transaction'] = results_table3['After Transaction'].round(5)

# Display the tabular results
print(results_table3)


                    Analysis  Before Transaction  After Transaction
0                   VaR (1%)            -0.00987           -0.00317
1                  CVaR (1%)            -0.01290           -0.00428
2                   VaR (5%)            -0.00535           -0.00132
3                  CVaR (5%)            -0.00833           -0.00274
4        Mean Return (Short)            -0.00252           -0.00175
5         Mean Return (Long)             0.00215            0.00078
6                     Median             0.00016            0.00000
7                 Quartile 1            -0.00136           -0.00000
8                 Quartile 3             0.00167            0.00034
9      Newey-West Std. Error             0.06477            0.05447
10  t-Statistic (Newey-West)            -1.57449           -0.89891
11         PT Test Statistic             0.00844            0.00031
12                  Skewness            -0.68981            1.72947
13                  Kurtosis             3.43205